# An illustration of SIMD packing and slot rotation in BGV/BFV FHE schemes for integer vector arithmetic

In [73]:
import sympy
from sympy import symbols, Poly, GF

def inverse_crt_interpolate(points, variable, modulus):
    """
    Computes the Lagrange Polynomial strictly in GF(p).
    Given points [(x0, y0), (x1, y1)...], find P(x) s.t. P(xi) = yi.
    Returns a SymPy Poly object.
    """
    final_poly = Poly(0, variable, domain=GF(modulus))
    
    xs = [pt[0] for pt in points]
    ys = [pt[1] for pt in points]
    k = len(points)
    
    for i in range(k):
        xi, yi = xs[i], ys[i]
        
        # Build Lagrange Basis Li(x)
        li_numerator = Poly(1, variable, domain=GF(modulus))
        li_denominator = 1
        
        for j in range(k):
            if i == j: continue
            
            xj = xs[j]
            # Numerator: (x - xj)
            term_poly = Poly(variable - xj, variable, domain=GF(modulus))
            li_numerator = li_numerator * term_poly
            
            # Denominator: (xi - xj)
            denom_term = (xi - xj) % modulus
            li_denominator = (li_denominator * denom_term) % modulus
            
        # Modular inverse of denominator
        denom_inv = pow(li_denominator, -1, modulus)
        
        # Add term: yi * Li(x)
        term = li_numerator.mul_ground((yi * denom_inv) % modulus)
        final_poly = final_poly + term
        
    return final_poly

def eval_poly_at_point(coeffs, x_val, p):
    """Evaluates a polynomial (given as list of coeffs) at a specific x."""
    result = 0
    for i, c in enumerate(coeffs):
        result = (result + c * pow(x_val, i, p)) % p
    return result

def apply_automorphism(coeffs, k, N, p):
    """
    Applies x -> x^k to the polynomial A(x).
    A_new(x) = A(x^k) mod (x^N + 1)
    """
    new_coeffs = [0] * N
    
    for i, c in enumerate(coeffs):
        if c == 0: continue
        
        # The term a_i * x^i becomes a_i * x^(i*k)
        power = i * k
        
        # Reduce x^power modulo (x^N + 1)
        # Rule: x^N = -1
        quotient = power // N
        remainder = power % N
        
        # If quotient is even, sign is +1. If odd, sign is -1.
        sign = pow(-1, quotient)
        
        # Add to the new bucket
        term_val = (c * sign) % p
        new_coeffs[remainder] = (new_coeffs[remainder] + term_val) % p
        
    return new_coeffs

def get_roots_of_unity(m, p):
    """Finds primitive m-th roots of unity modulo p."""
    roots = []
    # Brute force search for roots (serves the purpose here but not efficient for very large parameters)
    for i in range(1, p):
        # 1. Check order: x^m = 1
        if pow(i, m, p) == 1:
            # 2. Check it's a primitive root of x^(m/2) + 1
            # i^(m/2) must be -1 (which is p-1)
            if pow(i, m // 2, p) == p - 1:
                roots.append(i)
    return roots


In [74]:
# Setup Parameters
m = 16       # Cyclotomic index
p = 17       # Modulus
N = m // 2   # Number of slots (8)
x = symbols('x')

print(f"--- SETUP: m={m}, p={p}, N={N} ---")

# A. Identify the Roots (The Slots)
roots = get_roots_of_unity(m, p)
roots = sorted(roots) # Sort to ensure deterministic slot ordering
print(f"Roots (Slots):        {roots}")

--- SETUP: m=16, p=17, N=8 ---
Roots (Slots):        [3, 5, 6, 7, 10, 11, 12, 14]


In [75]:
# B. Define Messages
msgs = [0, 1, 2, 3, 4, 5, 6, 7]
print(f"Original Message:     {msgs}")

# C. Encode (Pack Messages into Polynomial)
points_msgs = list(zip(roots, msgs))
poly_sym = inverse_crt_interpolate(points_msgs, x, p)

# Convert SymPy Poly to simple list of coefficients [c0, c1, c2...]
# We extract explicitly to ensure we get exactly N coefficients
coeffs = [int(poly_sym.coeff_monomial(x**i)) % p for i in range(N)]
print(f"Encoded Poly Coeffs:  {coeffs}")

Original Message:     [0, 1, 2, 3, 4, 5, 6, 7]
Encoded Poly Coeffs:  [12, 12, 0, 15, 0, 11, 0, 11]


In [76]:
# D. Apply Galois Automorphism (The Rotation)
k = 3
print(f"\n--- Applying Automorphism x -> x^(k={k}) ---")

# 1. Visualize how roots shuffle physically
# We calculate r^k mod p for every root r
shuffled_roots = [pow(r, k, p) for r in roots]
print(f"Shuffled Roots:         {shuffled_roots}")

# 2. Apply automorphism to polynomial coefficients
rotated_coeffs = apply_automorphism(coeffs, k, N, p)
print(f"Rotated Poly Coeffs:  {rotated_coeffs}")


--- Applying Automorphism x -> x^(k=3) ---
Shuffled Roots:         [10, 6, 12, 3, 14, 5, 11, 7]
Rotated Poly Coeffs:  [12, 2, 0, 12, 0, 11, 0, 6]


In [77]:
# E. Decode (Evaluate at original roots)
# Note: We evaluate the NEW polynomial at the OLD roots
new_msgs = [eval_poly_at_point(rotated_coeffs, r, p) for r in roots]

# F. Analysis
print("\n--- Data Movement Analysis ---")
print(f"{'Slot':<5} | {'Original (Root)':<15} | {'Moved To (Root^k)':<20} | {'New Value'}")
print("-" * 60)
for i in range(N):
    original_root = roots[i]
    target_root = shuffled_roots[i] # Where the data from this slot WENT
    
    # To understand the vector change:
    # The value at Slot i is now what WAS at the slot corresponding to target_root
    print(f"{i:<5} | {original_root:<15} | {target_root:<20} | {new_msgs[i]}")

print("\nDirect Comparison:")
print(f"Old: {msgs}")
print(f"New: {new_msgs}")


--- Data Movement Analysis ---
Slot  | Original (Root) | Moved To (Root^k)    | New Value
------------------------------------------------------------
0     | 3               | 10                   | 4
1     | 5               | 6                    | 2
2     | 6               | 12                   | 6
3     | 7               | 3                    | 0
4     | 10              | 14                   | 7
5     | 11              | 5                    | 1
6     | 12              | 11                   | 5
7     | 14              | 7                    | 3

Direct Comparison:
Old: [0, 1, 2, 3, 4, 5, 6, 7]
New: [4, 2, 6, 0, 7, 1, 5, 3]


## Cyclic rotations

In [78]:
def get_generator_orbit(m, g):
    """
    Generates the orbit of g modulo m.
    If g is a generator, this list should contain all integers 
    coprime to m.
    """
    orbit = []
    current = 1 # g^0
    
    # We loop until we wrap back around to 1
    while True:
        orbit.append(current)
        current = (current * g) % m
        if current == 1:
            break
    return orbit

# Configuration for a Prime m
m = 17 
g = 3   # 3 is a primitive root modulo 17

print(f"--- Group Structure for m={m} ---")
print(f"Generator g = {g}")

# Calculate the cycle
orbit = get_generator_orbit(m, g)

print(f"\nOrder of Slots (Powers of g):")
print(orbit)

print(f"\nTotal Slots Covered: {len(orbit)}")
print(f"Is this a full cycle? {len(orbit) == m-1}")

# simulating the shift
print("\n--- Simulating Rotation (x -> x^g) ---")
# If we are at index `orbit[i]`, multiplying by g moves us to `orbit[i+1]`
for i in range(5): # Show first 5 steps
    current_idx = orbit[i]
    next_idx = (current_idx * g) % m
    print(f"Slot {i} (Index {current_idx:2d}) * {g} -> Index {next_idx:2d} (which is Slot {i+1})")

--- Group Structure for m=17 ---
Generator g = 3

Order of Slots (Powers of g):
[1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6]

Total Slots Covered: 16
Is this a full cycle? True

--- Simulating Rotation (x -> x^g) ---
Slot 0 (Index  1) * 3 -> Index  3 (which is Slot 1)
Slot 1 (Index  3) * 3 -> Index  9 (which is Slot 2)
Slot 2 (Index  9) * 3 -> Index 10 (which is Slot 3)
Slot 3 (Index 10) * 3 -> Index 13 (which is Slot 4)
Slot 4 (Index 13) * 3 -> Index  5 (which is Slot 5)


In [79]:
# What if we want to Rotate by 5 steps at once?
shift_amount = 5

# We compute the specific automorphism exponent k
# k = g^5 mod m
k_rot5 = pow(g, shift_amount, m) 

print(f"\n--- Simulating Rotation by {shift_amount} (x -> x^{k_rot5}) ---")
# Note: k_rot5 happens to be 5 because 3^5 = 243, and 243 % 17 = 5.
# This coincidence is specific to these numbers.

start_slot = 0
current_idx = orbit[start_slot] # Value at Slot 0 (which is 1)

# Apply the automorphism math: Index * k mod m
new_idx = (current_idx * k_rot5) % m 

print(f"Start at Slot {start_slot} (Value {current_idx})")
print(f"Apply x^{k_rot5} -> New Value is {new_idx}")

# Check where this new value lives in our ordered list
target_slot = orbit.index(new_idx)

print(f"Value {new_idx} corresponds to Slot {target_slot}")
print(f"Result: Instant jump from Slot {start_slot} to Slot {target_slot}")


--- Simulating Rotation by 5 (x -> x^5) ---
Start at Slot 0 (Value 1)
Apply x^5 -> New Value is 5
Value 5 corresponds to Slot 5
Result: Instant jump from Slot 0 to Slot 5


## Python Illustration: The Two-Row Matrix

In [80]:
def generate_row(m, start_val, g, length):
    row = []
    current = start_val
    for _ in range(length):
        row.append(current)
        current = (current * g) % m
    return row

# Parameters
m = 32
N = m // 2  # 16 slots total
row_len = N // 2 # 8 columns

# 1. Build the Matrix Structure
# Generator g=5 creates the cycle for the rows
g = 5 
# Generator h=-1 (31) jumps between rows

# Row 1 starts at 1
row1 = generate_row(m, 1, g, row_len)
# Row 2 starts at -1 (which is m-1)
row2 = generate_row(m, m-1, g, row_len)

print(f"--- Power-of-Two Structure (m={m}) ---")
print(f"Row 1 (Powers of 5):      {row1}")
print(f"Row 2 (Powers of 5 * -1): {row2}")

# 2. Simulate Rotation by 1 (x -> x^5)
print("\n--- Apply x -> x^5 (Shift within Rows) ---")
# If we multiply every index by 5, where does it go?
rotated_row1 = [(x * 5) % m for x in row1]
rotated_row2 = [(x * 5) % m for x in row2]

print(f"Row 1 moved to:           {rotated_row1}")
print(f"Row 2 moved to:           {rotated_row2}")

# Notice: The value 1 (Row1, Col0) moved to 5 (Row1, Col1).
# The rows rotated independently. Data did not cross rows.

# 3. Simulate Swap (x -> x^-1)
print("\n--- Apply x -> x^-1 (Swap Rows) ---")
# Multiply by -1 (m-1)
swapped_row1 = [(x * (m-1)) % m for x in row1]

print(f"Row 1 moved to:           {swapped_row1}")
print(f"Does this match Row 2?    {swapped_row1 == row2}")

--- Power-of-Two Structure (m=32) ---
Row 1 (Powers of 5):      [1, 5, 25, 29, 17, 21, 9, 13]
Row 2 (Powers of 5 * -1): [31, 27, 7, 3, 15, 11, 23, 19]

--- Apply x -> x^5 (Shift within Rows) ---
Row 1 moved to:           [5, 25, 29, 17, 21, 9, 13, 1]
Row 2 moved to:           [27, 7, 3, 15, 11, 23, 19, 31]

--- Apply x -> x^-1 (Swap Rows) ---
Row 1 moved to:           [31, 27, 7, 3, 15, 11, 23, 19]
Does this match Row 2?    True


## Comprehensive Python Walkthrough

In [81]:
# 1. SETUP PARAMETERS
m = 16       # Cyclotomic index (Power of Two)
p = 17       # Modulus
N = m // 2   # Number of slots

# distinct from the Galois generators '5' and '-1' used later for ordering.
zeta = 3     # The Field Generator: 3^16 = 1 mod 17. 
             # We use this generator to calculate the actual root values.

print(f"--- SETUP: m={m} (Power of 2), p={p} ---")

# 2. ORGANIZE ROOTS (The "Batch" Ordering)
# Standard FHE ordering uses generators g=5 and h=-1
# Row 1: Powers of 5 [1, 5, 9, 13]
row1_exponents = [pow(5, i, m) for i in range(N // 2)]
# Row 2: Powers of 5 * -1 [15, 11, 7, 3]
row2_exponents = [(e * -1) % m for e in row1_exponents]

ordered_exponents = row1_exponents + row2_exponents
roots = [pow(zeta, e, p) for e in ordered_exponents]

print(f"Row 1 Indices: {row1_exponents}")
print(f"Row 2 Indices: {row2_exponents}")
print(f"Ordered Roots: {roots}")

# 3. ENCODE DATA
# We pack [0..3] into Row 1, and [4..7] into Row 2
msgs = [0, 1, 2, 3, 4, 5, 6, 7]
print(f"\nOriginal Data:      {msgs}")

# Pack into polynomial
points_msgs = list(zip(roots, msgs))
poly_sym = inverse_crt_interpolate(points_msgs, symbols('x'), p)
coeffs = [int(poly_sym.coeff_monomial(symbols('x')**i)) % p for i in range(N)]

# 4. PERFORM ROTATE LEFT (x -> x^5)
# This should rotate 'Left' within each row
k_rot = 5
left_rotated_coeffs = apply_automorphism(coeffs, k_rot, N, p)

# Decode
decoded_rot = [eval_poly_at_point(left_rotated_coeffs, r, p) for r in roots]
print(f"\n--- After Rotate Left (x -> x^5) ---")
print(f"Decoded Data:       {decoded_rot}")
print("Observation:        [0,1,2,3] -> [1,2,3,0] (Row 1 Rotated Left by 1)")
print("                    [4,5,6,7] -> [5,6,7,4] (Row 2 Rotated Left by 1)")

# 5. PERFORM ROW SWAP (x -> x^-1)
# This should swap Row 1 and Row 2 completely
k_swap = pow(-1, 1, m) # m-1 = 15
swapped_coeffs = apply_automorphism(coeffs, k_swap, N, p)

# Decode
decoded_swap = [eval_poly_at_point(swapped_coeffs, r, p) for r in roots]
print(f"\n--- After Swap (x -> x^-1) ---")
print(f"Decoded Data:       {decoded_swap}")
print("Observation:        Rows swapped positions completely.")

# 6. PERFORM ROTATE RIGHT (x -> x^(5^-1))
# To rotate right, we use the inverse of the generator modulo m
# g = 5. We need g_inv such that 5 * g_inv = 1 mod 16.
# 5 * 13 = 65 = (4*16) + 1. So g_inv = 13.
k_right = pow(5, -1, m) # 13
right_rot_coeffs = apply_automorphism(coeffs, k_right, N, p)

# Decode
decoded_right = [eval_poly_at_point(right_rot_coeffs, r, p) for r in roots]
print(f"\n--- After Rotate Right (x -> x^{k_right}) ---")
print(f"Decoded Data:       {decoded_right}")
print("Observation:        [0,1,2,3] -> [3,0,1,2] (Row 1 Rotated Right by 1)")
print("                    [4,5,6,7] -> [7,4,5,6] (Row 2 Rotated Right by 1)")


--- SETUP: m=16 (Power of 2), p=17 ---
Row 1 Indices: [1, 5, 9, 13]
Row 2 Indices: [15, 11, 7, 3]
Ordered Roots: [3, 5, 14, 12, 6, 7, 11, 10]

Original Data:      [0, 1, 2, 3, 4, 5, 6, 7]

--- After Rotate Left (x -> x^5) ---
Decoded Data:       [1, 2, 3, 0, 5, 6, 7, 4]
Observation:        [0,1,2,3] -> [1,2,3,0] (Row 1 Rotated Left by 1)
                    [4,5,6,7] -> [5,6,7,4] (Row 2 Rotated Left by 1)

--- After Swap (x -> x^-1) ---
Decoded Data:       [4, 5, 6, 7, 0, 1, 2, 3]
Observation:        Rows swapped positions completely.

--- After Rotate Right (x -> x^13) ---
Decoded Data:       [3, 0, 1, 2, 7, 4, 5, 6]
Observation:        [0,1,2,3] -> [3,0,1,2] (Row 1 Rotated Right by 1)
                    [4,5,6,7] -> [7,4,5,6] (Row 2 Rotated Right by 1)
